In [ ]:
import sys
import snntorch as snn
import torch
import nir
from snntorch.export_nir import export_to_nir
from snntorch.import_nir import import_from_nir

### Generating an example NIR Graph from snntorch model:

In [2]:
# 2D sample data 
sample_data = torch.randn(1, 1, 28, 28)  #input image size is 28x28 and 1 channel #the dimension here is batch_size*number_of_channels*height*width

class NetWithAvgPool(torch.nn.Module):
    def __init__(self):
        super(NetWithAvgPool, self).__init__()
        self.conv1 = torch.nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.lif1 = snn.Leaky(beta=0.9, init_hidden=True)
        self.fc1 = torch.nn.Linear(28*28*16 // 4, 500)  # Adjusting input size after pooling layer
        self.lif2 = snn.Leaky(beta=0.9, init_hidden=True, output=True)

    def forward(self, x):
        x = torch.nn.functional.avg_pool2d(self.conv1(x), kernel_size=2, stride=2)  #  kernel_size=2 and stride=2 for avg_pool2d
        x = x.view(-1, 28*28*16 // 4)  # Adjusting the view based on the output shape after pooling
        x = self.lif1(x)
        x = self.fc1(x)
        x = self.lif2(x)
        return x

net_with_avgpool = NetWithAvgPool()
nir_graphtest = export_to_nir(net_with_avgpool, sample_data, ignore_dims=[0])

In [ ]:
nir_graphtest.infer_types()

In [ ]:
nir.write(f"./snnTorch/examples/testconv2d+avgpool.nir", nir_graphtest)

In [3]:
nir_graph = nir.read("testconv2d+avgpool.nir")

In [ ]:
#dict_keys(['conv1', 'fc1', 'input', 'lif1', 'lif2', 'output'])
nir_graph.nodes['lif1']

In [4]:
nir_graph.nodes['lif1']

LIF(tau=0.0009999997615814777, r=9.999997615814776, v_leak=0.0, v_threshold=1.0, input_type={'input': array([], dtype=float64)}, output_type={'output': array([], dtype=float64)}, metadata={})

In [ ]:
nir_graph.nodes.keys()

In [ ]:
nir_graph.edges